# Real or Not? NLP with Disaster Tweets
Predict which Tweets are about real disasters and which ones are not

we have files which include a train data,test data and sample submission file ,
We will be predicting whether a given tweet is about a real disaster or not. If so, predict a 1. If not, predict a 0.
using natural language processing.

here we will split training data into train and test split ,to test our model performance,post that we use our machine learning model to test on unseen data,that is test set.

In [2]:
import pandas as pd
import numpy  as np

In [3]:
df=pd.read_csv('train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [5]:
df.keyword.value_counts(dropna=False)

NaN                      61
fatalities               45
deluge                   42
armageddon               42
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 222, dtype: int64

In [6]:
df.location.value_counts(dropna=False)

NaN                     2533
USA                      104
New York                  71
United States             50
London                    45
                        ... 
LiVE MÌS                  1
Bagalkote Karnataka        1
Detroit, Michigan          1
Uganda                     1
Concord, N.C.              1
Name: location, Length: 3342, dtype: int64

In [7]:
df.drop(labels='location',inplace=True,axis=1)

In [8]:
df.isnull().sum()

id          0
keyword    61
text        0
target      0
dtype: int64

In [9]:
df[df['target']==1].isnull().sum()

id          0
keyword    42
text        0
target      0
dtype: int64

### as we can see,out of 61 null values,42 null values in keywords are in Taregt=1,that means where the diisaster has happened,so we will replace the 42 null values with mode count,remaining will drop which are not necessary for the analysis

In [10]:
df['keyword'].fillna(df[df['target']==1].keyword.mode(), inplace=True)

In [11]:
df.isnull().sum()

id          0
keyword    58
text        0
target      0
dtype: int64

In [12]:
df[df['target']==0].isnull().sum()

id          0
keyword    19
text        0
target      0
dtype: int64

In [13]:
df[df['target']==1].isnull().sum()

id          0
keyword    39
text        0
target      0
dtype: int64

In [14]:
df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

### We can see we have classes which are balanced,and good for analysis

In [15]:
df.dropna(inplace=True)

In [16]:
df.isnull().sum()

id         0
keyword    0
text       0
target     0
dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7555 entries, 0 to 7582
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       7555 non-null   int64 
 1   keyword  7555 non-null   object
 2   text     7555 non-null   object
 3   target   7555 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 295.1+ KB


## Lets seperate target and independent variables

In [18]:
X=df.drop('target',axis=1)
y=df['target']

In [19]:
X

,id,keyword,text
0,1,derailment,Our Deeds are the Reason of this #earthquake M...
1,4,outbreak,Forest fire near La Ronge Sask. Canada
2,5,wreckage,All residents asked to 'shelter in place' are ...
31,48,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...
32,49,ablaze,We always try to bring the heavy. #metal #RT h...
...,...,...,...
7578,10830,wrecked,@jt_ruff23 @cameronhacker and I wrecked you both
7579,10831,wrecked,Three days off from work and they've pretty mu...
7580,10832,wrecked,#FX #forex #trading Cramer: Iger's 3 words tha...
7581,10833,wrecked,@engineshed Great atmosphere at the British Li...


In [20]:
X.drop(labels='id',inplace=True,axis=1)

In [21]:
y

0       1
1       1
2       1
31      1
32      0
       ..
7578    0
7579    0
7580    0
7581    0
7582    0
Name: target, Length: 7555, dtype: int64

In [22]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [23]:
X.head()

,keyword,text
0,derailment,Our Deeds are the Reason of this #earthquake M...
1,outbreak,Forest fire near La Ronge Sask. Canada
2,wreckage,All residents asked to 'shelter in place' are ...
31,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...
32,ablaze,We always try to bring the heavy. #metal #RT h...


In [24]:
# Removing punctuations
X=X.iloc[:,0:2]
X.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(2)]
new_Index=[str(i) for i in list1]
X.columns= new_Index
X.head(5)

,0,1
0,derailment,Our Deeds are the Reason of this earthquake M...
1,outbreak,Forest fire near La Ronge Sask Canada
2,wreckage,All residents asked to shelter in place are ...
31,ablaze,bbcmtd Wholesale Markets ablaze http t co l...
32,ablaze,We always try to bring the heavy metal RT h...


In [25]:
# Convertng headlines to lower case
for index in new_Index:
    X[index]=X[index].str.lower()
X.head(1)

,0,1
0,derailment,our deeds are the reason of this earthquake m...


In [26]:
' '.join(str(x) for x in X.iloc[1,0:2])

'outbreak forest fire near la ronge sask  canada'

In [27]:
tweet = []### Combining both the columns
for row in range(0,len(X.index)):
    tweet.append(' '.join(str(x) for x in X.iloc[row,0:3]))

In [28]:
tweet[0]

'derailment our deeds are the reason of this  earthquake may allah forgive us all'

In [29]:
len(tweet)

7555

In [30]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(tweet)):
    review = re.sub('[^a-zA-Z]', ' ', tweet[i])
    review = review.lower()
    review = review.split() 
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [31]:
corpus[0]

'derail deed reason earthquak may allah forgiv us'

In [32]:
len(corpus)

7555

In [33]:
## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(corpus)

In [34]:
countvector

CountVectorizer(ngram_range=(2, 2))

In [35]:
traindataset

<7555x60404 sparse matrix of type '<class 'numpy.int64'>'
	with 84651 stored elements in Compressed Sparse Row format>

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(traindataset, y, test_size=0.33, random_state=42)

In [37]:
X_train.shape

(5061, 60404)

In [38]:
from sklearn.ensemble import RandomForestClassifier

# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [39]:
predictions = randomclassifier.predict(X_test)
predictions

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [40]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test,predictions)
print(matrix)
score=accuracy_score(y_test,predictions)
print(score)
report=classification_report(y_test,predictions)
print(report)

[[1413   30]
 [ 677  374]]
0.7165196471531676
              precision    recall  f1-score   support

           0       0.68      0.98      0.80      1443
           1       0.93      0.36      0.51      1051

    accuracy                           0.72      2494
   macro avg       0.80      0.67      0.66      2494
weighted avg       0.78      0.72      0.68      2494



In [41]:
from sklearn.naive_bayes import MultinomialNB

In [42]:
mnb=MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

In [43]:
y_pred = mnb.predict(X_test)
y_pred

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [44]:
cm=confusion_matrix(y_test,y_pred)
print(cm)
acc=accuracy_score(y_test,y_pred)
print(acc)
rep=classification_report(y_test,y_pred)
print(rep)

[[868 575]
 [211 840]]
0.6848436246992783
              precision    recall  f1-score   support

           0       0.80      0.60      0.69      1443
           1       0.59      0.80      0.68      1051

    accuracy                           0.68      2494
   macro avg       0.70      0.70      0.68      2494
weighted avg       0.72      0.68      0.69      2494



In [45]:
from sklearn.model_selection import cross_val_score
nb_Kfold_accu = cross_val_score(estimator = mnb, X = X_train, y = y_train, cv = 10)
nb_Kfold_accu=nb_Kfold_accu.mean()
print("Accuracy: {:.2f} %".format(nb_Kfold_accu*100))

Accuracy: 68.45 %


### We observed both models Multinomial and random forest ,howver random forest gave us the better results compared to the other,with accuracy of 72 %

### Importing TEST Data

In [46]:
test=pd.read_csv('test.csv')
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [47]:
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [48]:
test.drop(labels=['location'],axis=1,inplace=True)

In [50]:
test.keyword.value_counts()

deluged                  23
rubble                   22
demolished               22
snowstorm                21
seismic                  21
                         ..
radiation%20emergency     5
fatalities                5
forest%20fire             5
inundation                4
epicentre                 1
Name: keyword, Length: 221, dtype: int64

### we will replace missing values with most number of repeated keywords,that is mode of the list

In [51]:
test['keyword'].fillna('deluged',inplace=True)

In [52]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       3263 non-null   int64 
 1   keyword  3263 non-null   object
 2   text     3263 non-null   object
dtypes: int64(1), object(2)
memory usage: 76.6+ KB


In [54]:
df2=test.copy()

### similarly cleaning out test data like training data for prediction

In [55]:
test.drop(labels=['id'],axis=1,inplace=True)

In [56]:
# Removing punctuations
test=test.iloc[:,0:2]
test.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list2= [j for j in range(2)]
new_Index1=[str(j) for j in list2]
test.columns= new_Index1
test.head(5)

,0,1
0,deluged,Just happened a terrible car crash
1,deluged,Heard about earthquake is different cities s...
2,deluged,there is a forest fire at spot pond geese are...
3,deluged,Apocalypse lighting Spokane wildfires
4,deluged,Typhoon Soudelor kills in China and Taiwan


In [57]:
' '.join(str(x) for x in test.iloc[1,0:2])

'deluged Heard about  earthquake is different cities  stay safe everyone '

In [58]:
test_tweet = []
for row in range(0,len(test.index)):
    test_tweet.append(' '.join(str(x) for x in test.iloc[row,0:2]))

In [59]:
test_tweet[1]

'deluged Heard about  earthquake is different cities  stay safe everyone '

In [60]:
ps = PorterStemmer()
test_corpus = []
for i in range(len(test_tweet)):
    review1 = re.sub('[^a-zA-Z]', ' ',test_tweet[i])
    review1 = review1.lower()
    review1 = review1.split() 
    review1 = [ps.stem(word) for word in review1 if not word in stopwords.words('english')]
    review1 = ' '.join(review1)
    test_corpus.append(review1)

In [61]:
test_corpus[0]

'delug happen terribl car crash'

### Prediction of  test data using the RF model

In [62]:
test_dataset = countvector.transform(test_corpus)
predict1 = randomclassifier.predict(test_dataset)
predict1

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [63]:
sub= pd.DataFrame({'id':df2.id,
                      'target':predict1})

sub.to_csv("outcome.csv",index=False,header = True)